In [11]:
import rasterio
import numpy as np
from affine import Affine
import shutil
import shapely

from rasterio.windows import Window

ModuleNotFoundError: No module named 'shapely'

In [10]:
path = '/work/OT/ai4usr/fournip/tmp/Toulouse.tif'

with rasterio.open(path, 'r+') as src:
    src.nodata = 0
    mask = src.dataset_mask()
    mask_polys = []
    for shape, value in rasterio.features.shapes(
        mask, mask=mask, connectivity=8, transform=src.transform
    ):
        mask_polys.append(shapely.geometry.shape(shape))
    self.raster_mask_poly = shapely.geometry.MultiPolygon(mask_polys)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0, 'width': 55600, 'height': 54062, 'count': 4, 'crs': CRS.from_epsg(32631), 'transform': Affine(0.5, 0.0, 358326.640592619,
       0.0, -0.5, 4843218.187181153)}


In [ ]:
output_transform = Affine.translation(dst_w, dst_n) * Affine.scale(res[0], -res[1])

In [ ]:
train_zone = intersections[16] # raster coords
train_zone_bounds = train_zone.bounds
# raster pixel coords
tile = rasterio.windows.from_bounds(
    *train_zone_bounds, 
    transform=raster.transform
).round_offsets().round_lengths()
tile_poly = shapely.Polygon(generate_polygon(train_zone_bounds))
tile_patch = plotting.patch_from_polygon(tile_poly, facecolor=facecolor, edgecolor=edgecolor)

area = 0
crop_size=512
while area <= 0 or area >= 1:
    cx = tile.col_off + np.random.randint(0, tile.width - crop_size + 1)
    cy = tile.row_off + np.random.randint(0, tile.height - crop_size + 1)
    # raster pixel coords
    crop = rasterio.windows.Window(cx, cy, crop_size, crop_size)
    left, bottom, right, top = rasterio.windows.bounds(crop, transform=raster.transform)
    crop_poly = shapely.Polygon(generate_polygon((left, bottom, right, top)))
    crop_patch = plotting.patch_from_polygon(crop_poly, facecolor=facecolor, edgecolor=edgecolor)
    intersection = shapely.intersection(crop_poly, train_zone)
    area = shapely.area(intersection) / shapely.area(crop_poly)
print(area)
train_zone_patch = plotting.patch_from_polygon(train_zone, facecolor=facecolor, edgecolor=edgecolor)
crop_img, crop_transform = rasterio.mask.mask(raster, shapes=[intersection], crop=True)


fig3, ax3 = plt.subplots()
ax3.add_patch(tile_patch)
ax3.add_patch(crop_patch)
ax3.add_patch(train_zone_patch)
rioplt.show(crop_img, ax=ax3, transform=crop_transform)
ax3.autoscale_view()